# BT2101 Introduction to Linear Regression

## 1 Goal:

In this notebook, we will explore multiple linear regression using:
* Gradient descent method
* Open-source package: `scikit-learn`

For the gradient descent method, you will:
* Use numpy to write functions
* Write a derivative function
* Write an output function
* Write a gradient descent function
* Add a constant column of 1's as intercept term
* Use the gradient descent function to get regression estimators

In [ ]:
# -*- coding:utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
from __future__ import division
%matplotlib inline

### 1.1 Summary of Multiple Linear Regression

From the lecture class, we know that a typical linear regression model of *N* observations and *p* predictors:

\begin{align}
Y &= \beta_0 + \sum_{j=1}^p x_j \beta_j + \varepsilon \\
&= X\beta + \varepsilon 
\end{align}

And we aim to **Minimize** the loss function:

$$RSS(\beta) = \varepsilon^T\varepsilon = \sum_{i=1}^{N}(y_i - \beta_0 - \sum_{j=1}^p x_j \beta_j)^2 $$

Remember that the predicted output is calculated by the dot product of features and weights.<br/>
Suppose we have: <br/>
Two features: $x_i = [2.0, 1.5]$ <br/>
and Weights: $\beta = [1.0, 1.0]$ <br/>
Then the predicted output: $ \hat{y}_i = 1.0\times2.0 + 1.0\times1.5 = 3.5 $

In [ ]:
def predict(feature_matrix, weight_vector):
    '''This function is used to predict outputs.
    
    Inputs:
    1) feature_matrix: A matrix of selected features;
    2) weight_vector: A vector of weights for selected features;
    
    Outputs:
    1) predict_vector: A vector of predicted outputs   
    
    '''
    
    prediction_vector = np.dot(feature_matrix, weight_vector)

    return prediction_vector

In [ ]:
# Let us have a test
features = np.array([[2.0, 1.5], [4.8, 5.2]])
weights = np.ones((2, 1))
predict(features, weights)

## 2 Gradient Descent
### 2.1 Computing Derivatives

How can we compute the derivatives of the loss function? <br/>
The loss function is calculated by summing up the squared errors over *N* data observations/rows:
$$RSS(\beta) = \sum_{i=1}^{N}(y_i - \beta_0 - \sum_{j=1}^p x_j \beta_j)^2 $$

Just doing simple math, you will find that the derivative of a sum equals to the sum of the derivatives. For a single observation *i*, the derivative with respect to feature $x_j$ for this observation *i* is $ -2\varepsilon_{i}x_{ij} $. So the derivative with respect to feature $x_j$ is $ -2\sum_{i=1}^N \varepsilon_{i}x_{ij} = -2\varepsilon^{T}x_j $. <br/>

In [ ]:
def derivative(feature_matrix, error_vector):
    '''This function is used to calculate the derivatives for features.
    Inputs:
    1) feature_matrix: Data matrix of features (j = 0,...,p)
    2) error_vector: A vector of errors of N observations    
    
    Outputs:
    1) derive: Derivative for this feature j    
    
    '''
    
    derive = -2 * np.dot(feature_matrix.T, error_vector)
    
    return derive

In [ ]:
# Let us have a test
features = np.array([[2.0, 1.0], [4.0, 3.0]])
weights = np.ones((2, 1))
true_outputs = np.array([[1.5], [2.5]])
predict_outputs = predict(features, weights)
errors = true_outputs - predict_outputs

print "True output is: " 
print true_outputs
print "Predicted output is: "
print predict_outputs
print "Error is: "
print errors
print "Derivative is: "
print derivative(features, errors)

### 2.2 Procedures for Gradient Descent

Step 0: Initialize $ \beta = (\beta_{0},\dots,\beta_{p}) $; <br/>
Step 1: Calculate $ \beta_{j} \leftarrow \beta_{j} - Stepsize\times(-2\varepsilon^{T}x_j) $, for j = 1,...,p; <br/>
Step 2: If not converged, go back to Step 1; <br/>
Step 3: Get $ \beta $.

In [ ]:
def OLS_GD(initial_weights, feature_matrix, true_output, step_size, tolerance, n_iter):
    '''This function is used to iteratively calculate OLS estimators for multiple regression model using Gradient Descent.
    Inputs:
    1) initial_weights: Initial regression coefficients
    2) feature_matrix: A matrix of selected features
    3) true_output: A vector of true outputs
    4) step_size: Size of step for each iteration of gradient search
    5) tolerance: Indicate converging condition
    6) n_iter: Maximum number of iterations
    
    Outputs:
    1) weights: OLS coefficients.    
    
    '''
    
    weights = np.array(initial_weights, dtype=np.float64)
    
    for i in range(n_iter):
        # Calculate predictions
        prediction = predict(feature_matrix, weights)
        
        # Calculate errors
        error = true_output - prediction
        
        # Calculating derivatives for weights
        derivative_vector = derivative(feature_matrix, error)
        
        # Updating weights
        weights -= step_size * derivative_vector        
        
        # Converging conditions: L2 norm for derivatives
        sum_squared_gradient = np.sum(np.power(derivative_vector,2))
        
        # If converging?
        if np.sqrt(sum_squared_gradient) < tolerance:
            return weights
    
    return weights


## 3 Convert Pandas DataFrame to Numpy Array

We need to: <br/>
1. Convert Pandas DataFrame into a Numpy Array/Matrix to do internal calculations;
2. Augment this Array/Matrix by adding 1's column in the first column, in order to calculate the intercept.

In [ ]:
def convert_data(df, feature_names, output_name):
    '''This function is used to convert pandas dataframe to numpy array/matrix, and augment it with 1's column as intercept.
    Inputs:
    1) df: Original data in the format of pandas dataframe
    2) feature_names: A list of names of selected features
    3) output_name: Name of selected outputs
    
    Outputs:
    1) feature_matrix: Augmented feature matrix
    2) output_vector: A vector of true outputs    
    
    '''    
    
    # Feature names of Augmented matrix 
    augment_feature_names = ['intercept'] + feature_names
    
    # Augmented feature matrix by adding constant 1's as intercept term, and reorder the feature matrix
    df['intercept'] = 1 
    feature_matrix = df[augment_feature_names]
    n, k = feature_matrix.shape # n: number of observations; k: number of weights
    
    # Convert selected feature matrix and output vector to Numpy Array
    feature_matrix = feature_matrix.values.reshape((n, k))
    output_vector = df[output_name].values.reshape((n, 1))

    return (feature_matrix, output_vector)

## 4 Load Dataset and Conduct OLS Estimations

Our dataset can be obtained from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Iris).

### 4.1 Attribute Information:

Attribute Information:

1. sepal length in cm 
2. sepal width in cm 
3. petal length in cm 
4. petal width in cm 
5. class: <br/>
-- Iris Setosa <br/>
-- Iris Versicolour <br/>
-- Iris Virginica


In [ ]:
# Import dataset into Python Pandas DataFrame
filepath = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
colnames = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
data = pd.read_csv(filepath, header=None, names=colnames, dtype={'sepal_length':np.float64, 'sepal_width':np.float64, 'petal_length':np.float64, 'petal_width':np.float64})
data.head(n=10)

In [ ]:
data.dropna() # Drop null values
data.describe()

In [ ]:
# Select features and output
feature_list =  ['sepal_width', 'petal_length']
output_label = 'sepal_length'

features = convert_data(data, feature_list, output_label)[0]
output = convert_data(data, feature_list, output_label)[1]

In [ ]:
features
output

Now set up initial parameters and try gradient descent:

In [ ]:
# Set up initial parameters
initial_weights = np.zeros((len(feature_list)+1, 1))
step_size = 7e-5 # Large step_size may lead to overflow
tolerance = 2e-9
iteration = 100000

In [ ]:
# Try OLS with gradient descent
coefficients = OLS_GD(initial_weights, features, output, step_size, tolerance, iteration)
print 'Intercept ','X1', 'X2'
coefficients

You can try different `initial_weights`, `step_size`, `tolerance`, or `iteration`.<br/>
You can also use different features to run regression model. <br/>
Note that large step_size may lead to overflow issue. <br/>
Small tolerance may increase the number of iterations, but will result in more accurate estimators.

### 4.2 Compare with OLS Expression

Remember in the lecture class, we show the regression expression for OLS estimations:
$$ \beta = (X^{T}X)^{-1}X^{T}Y $$

In [ ]:
def OLS_Expression(feature_matrix, true_output):
    '''This function is used to calculate OLS coefficients using standard expression.
    Inputs:
    1) feature_matrix: A matrix of selected features
    2) true_output: A vector of true outputs
        
    Outputs:
    1) weights: OLS coefficients
    
    '''
    
    XTX = np.dot(feature_matrix.T, feature_matrix)
    XTY = np.dot(feature_matrix.T, true_output)
    
    weights = np.dot(np.linalg.inv(XTX), XTY)
    
    return weights
    

In [ ]:
coefficients = OLS_Expression(features, output)
print 'Intercept ','X1', 'X2'
print coefficients

Compare OLS using gradient descent with OLS using expression. They are very close to each other.

## 5 Try Open-Source Package
### 5.1 Scikit-Learn
The package `scikit-learn` can be found at http://scikit-learn.org/stable/index.html. <br/>
Please install the package first.

In [ ]:
from sklearn import linear_model

In [ ]:
reg = linear_model.LinearRegression(fit_intercept=True)
reg.fit(X=data[feature_list], y=data[output_label])

In [ ]:
print reg.intercept_, reg.coef_

Compare with OLS using gradient descent and OLS using expression. They are very close to each other.

Then we can calculate $ R^2 $, which measures goodness-of-fit of regression model.

In [ ]:
print 'R_squared is: ', reg.score(X=data[feature_list], y=data[output_label])

In [ ]:
# Plot outputs
plt.scatter(data[feature_list[0]], data[output_label], color='black')

In [ ]:
plt.plot(data[feature_list[1]], reg.predict(data[feature_list]), color='blue', linewidth=1)

More about Multiple Linear Regression can be found at http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html.

## 6 Questions

#### How to use Stochastic Gradient Descent to do gradient search?
Hint: The procedure for stochastic gradient descent: <br/>
* Initialize all weights
* Repeat until convergence or stopping condition is satisfied: <br/>
    Randomly shuffle the dataset <br/>
    For each piece of data in the dataset: <br/>
    (1) Calculate the derivative of one piece of data <br/>
    (2) Update the weights by $stepsize \times derivative$ <br/>
    
Online resources: https://en.wikipedia.org/wiki/Stochastic_gradient_descent

#### How to use Mini-Batch Gradient Descent to do gradient search?

Hint: The general procedure is similar to stochastic descent, except that mini-batch uses multiple pieces of data to calculate derivative and update the weights, rather than one piece of data.

Online resources: http://cs229.stanford.edu/notes/cs229-notes1.pdf

## 7 References

[1] Jason Brownlee, 2018, [Machine Learning Algorithms from Scratch with Python](https://machinelearningmastery.com/machine-learning-algorithms-from-scratch/). <br/>
[2] Peter Harrington, 2012. Machine Learning in Action. Shelter Island, NY: Manning Publications Co.